# Rolling Method Sanity Check: Study Description & Results

## **Study Overview**

The goal of this experiment was to **benchmark different rolling aggregation strategies** for time series features in a tabular modeling task (e.g., sales or trading prediction). Four rolling methods were evaluated for their effect on model performance across training, validation, and test splits.

**Methods Compared:**

* Rolling Mean (`rolling_mean`)
* Rolling Median (`rolling_median`)
* Rolling Weighted Mean (`rolling_weighted_mean`)
* Rolling Winsorized Mean (`rolling_winsorized_mean`)

**Metric:**
Model performance was assessed using an error metric (likely RMSLE or similar) on three dataset splits: train, validation, and test.

---

## **Experiment Steps**

1. **Data Preparation:**

   * Data was split into three periods: train, validation, and test.
2. **Rolling Feature Engineering:**

   * Each rolling method was applied to generate features.
3. **Model Training:**

   * A consistent model pipeline was trained for each feature variant.
4. **Performance Logging:**

   * Errors were logged for each split and rolling method.
5. **Analysis:**

   * Differences and stability across splits were analyzed, with visualizations (e.g., heatmaps).

---

## **Results Table**

| Rolling Method          | Train Error | Val Error  | Test Error |
| ----------------------- | ----------- | ---------- | ---------- |
| Rolling Mean            | 0.3917      | 0.3967     | 0.3825     |
| Rolling Median          | 0.3974      | 0.4046     | 0.3850     |
| Rolling Weighted Mean   | **0.3876**  | **0.3892** | 0.3896     |
| Rolling Winsorized Mean | 0.3920      | 0.3960     | **0.3797** |

**Stability Across Splits (Mean Absolute Drift):**

| Method                  | Mean Drift | Train-Test Abs Diff |
| ----------------------- | ---------- | ------------------- |
| Rolling Mean            | 0.0071     | 0.0093              |
| Rolling Median          | 0.0098     | 0.0124              |
| Rolling Weighted Mean   | **0.0018** | **0.0020**          |
| Rolling Winsorized Mean | 0.0082     | 0.0122              |

---

## **Key Findings & Interpretation**

* **Best Test Error:**

  * The **Winsorized Mean** achieved the lowest error on the test split, but not by a large margin.
* **Best Stability/Generalization:**

  * The **Weighted Mean** was by far the most stable, with nearly identical performance across all splits. Its train–test drift was substantially lower than all other methods.
* **Rolling Mean** and **Median** performed worse than either advanced method, with both higher error and more drift.

**Summary Table:**

| Method                  | Best For                  | Drawbacks                  |
| ----------------------- | ------------------------- | -------------------------- |
| Rolling Weighted Mean   | Stability, generalization | Slightly higher test error |
| Rolling Winsorized Mean | Test error                | Higher drift               |
| Rolling Mean            | Simplicity                | Not best on any metric     |
| Rolling Median          | Outlier resistance        | Most unstable              |

---

## **Conclusion**

* **Rolling Weighted Mean** is recommended for **robustness and stability**—it generalizes well and minimizes overfitting.
* **Winsorized Mean** may provide the best absolute test result, but at the cost of less stability.
* **Classic mean/median** approaches are outperformed by the above for both stability and test performance.

---

## **Next Steps & Suggestions**

* **If the priority is reliability and robustness in production or real-world deployment, stick with Rolling Weighted Mean.**
* For maximum accuracy on the specific test split (e.g., competitions), consider Winsorized Mean, but monitor for overfitting or distribution shifts.

---




In [4]:
# Store Sales Forecasting - Real Dataset with RMSLE Optimization
import jupyter

import os
import time
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt



from scipy.stats import mstats
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split

from src.utils.system import boot,notify
from src.experiments.experiment_tracker import ExperimentTracker

boot()

experience_name = "sanity-check__evaluate_efficiency_of_averaging_policies"
target_date = "2025-06-01"
experiment_tracker = ExperimentTracker(experience_name)
config={
    "dataset":"kaggle-sales-prediction",
    "model": "xgb.XGBRegressor",
    "rolling_window":"14,21"
}

features = [
        'store_nbr', 'family', 'dayofweek', 'month', 'day', 'week', 'is_holiday',
        'transactions', 'lag_21', 'lag_14', 'trans_lag_21', 'trans_lag_14',
        'rolling_mean_21', 'rolling_std_21', 'rolling_mean_14', 'rolling_std_14',
        'trans_roll_mean_21', 'trans_roll_mean_14',
        'onpromo_lag_21', 'onpromo_mean_14'
]

run_settings={
    "n_estimators":750, 
    "learning_rate":0.05, 
    "max_depth":6, 
    "random_state":42,
    "regime": "rolling_mean",
    "features": features.copy()
}
run_settings["features"].sort()


#def save_run(
#        self,
#        config: Dict,
#        results: Dict,
#        target_date: str,
#        run_settings: Dict,
#        files: Optional[Dict] = None
#    ) -> None:
#        """

In [6]:
# Store Sales Forecasting - Real Dataset with RMSLE Optimization
import jupyter
import os
import time
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt




from scipy.stats import mstats
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split

from src.utils.system import boot,notify


boot()


class ExecutionTimeTracker:
    def __init__(self,name):
        self.name = name
        self.start = time.time()
        self.end = None
        self.signature = f"[ExecutionTimeTracker]::{self.name} -"
        print(f"{self.signature} Start.")

    def done(self):
        if self.end == None:
            self.end =time.time()
            self.duration = self.end -self.start
            
        print(f"{self.signature} Complete. Took {int(self.end-self.start)}s to complete.")
        print('')
        
# REGIME FUNCTIONS =================================================
def rolling_mean(x):
    if len(x) == 0 or np.all(np.isnan(x)):
        return np.nan
    return np.nanmean(x)

def rolling_median(x):
    if len(x) == 0 or np.all(np.isnan(x)):
        return np.nan
    return np.nanmedian(x)

def rolling_weighted_mean(x): 
    if len(x) == 0 or np.all(np.isnan(x)):
        return np.nan
    weights = np.arange(1, len(x)+1)
    valid = ~np.isnan(x)
    if np.sum(valid) == 0:
        return np.nan
    return np.average(x[valid], weights=weights[valid])

def rolling_winsorized_mean(x):
    if len(x) < 2 or np.all(np.isnan(x)):
        return np.nan
    x_clean = x[~np.isnan(x)]
    if len(x_clean) < 2:
        return np.nan
    return mstats.winsorize(x_clean, limits=(0.1, 0.1)).mean()

# REGIME FUNCTIONS REGISTRY =======================================
regimes = {
    'rolling_mean': rolling_mean,
    'rolling_median': rolling_median,
    'rolling_weighted_mean': rolling_weighted_mean,
    'rolling_winsorized_mean': rolling_winsorized_mean,
}

# Feature engineering pipeline with regime-based rolling =========
def add_lag_rolling(df, train_func, train_name, lag_days=[14, 21], rolling_windows=[14, 21],mode="train"):
    FOLDER = 'data/experiments/kaggle-sales-prediction/'
    file_path = os.path.join(FOLDER, train_name+'__21_'+mode+'.csv')
    exists = os.path.exists(file_path)
    
    if exists:
        cached_df = pd.read_csv(file_path, parse_dates=['date'])
        if not cached_df.empty:
            print(f"Loaded cached features from {file_path}")
            return cached_df
    
    df = df.sort_values(['store_nbr', 'family', 'date'])

    for lag in [14, 21]:
        df[f'lag_{lag}'] = df.groupby(['store_nbr', 'family'])['sales'].shift(lag)
        df[f'trans_lag_{lag}'] = df.groupby(['store_nbr'])['transactions'].shift(lag)

    for window in [14, 21]:
        df[f'rolling_mean_{window}'] = df.groupby(['store_nbr', 'family'])['sales'].transform(lambda x: x.shift(1).rolling(window, min_periods=1).apply(train_func, raw=True))#.shift(1)#.rolling(window, min_periods=1).mean()
        df[f'rolling_std_{window}'] = df.groupby(['store_nbr', 'family'])['sales'].shift(1).rolling(window, min_periods=1).std()
        df[f'trans_roll_mean_{window}'] = df.groupby(['store_nbr'])['transactions'].transform(lambda x: x.shift(1).rolling(window, min_periods=1).apply(train_func, raw=True))#.shift(1)#.rolling(window, min_periods=1).mean()

    df['onpromo_lag_21'] = df.groupby(['store_nbr', 'family'])['onpromotion'].shift(7)
    df['onpromo_mean_14'] = df.groupby(['store_nbr', 'family'])['onpromotion'].transform(lambda x: x.shift(1).rolling(14, min_periods=1).apply(train_func, raw=True))#.shift(1)#.rolling(14, min_periods=1).mean()

    df.to_csv(file_path, index=False)
    return df

# GENERATE BASE TRAIN/TEST DATASET ======================================


def preprocess_dataset():
    FOLDER = 'data/experiments/kaggle-sales-prediction/'
    _train = pd.read_csv(FOLDER+'train.csv', parse_dates=['date'])
    _test = pd.read_csv(FOLDER+'test.csv', parse_dates=['date'])
    _stores = pd.read_csv(FOLDER+'stores.csv')
    _holidays = pd.read_csv(FOLDER+'holidays_events.csv', parse_dates=['date'])
    _transactions = pd.read_csv(FOLDER+'transactions.csv', parse_dates=['date'])
    _df = pd.concat([_train,_test]).sort_values(by="date")
    _separator = _test.iloc[0]['date']
    def preprocess(df,stores,holidays,transactions):
        train = df
        
        # 2. Merge External Features
        train = train.merge(stores, on='store_nbr', how='left')
        train = train.merge(transactions, on=['date', 'store_nbr'], how='left')

        holidays = holidays[(holidays['locale'] == 'National') & (holidays['transferred'] == False)]
        holidays = holidays[['date']].drop_duplicates()
        holidays['is_holiday'] = 1
        train = train.merge(holidays, on='date', how='left')
        train['is_holiday'] = train['is_holiday'].fillna(0)

        # Merge onpromotion before encoding 'family'
        #train = train.merge(test[['date', 'store_nbr', 'family', 'onpromotion']],
        #                    on=['date', 'store_nbr', 'family'], how='left')

        # Clean up column names
        if 'onpromotion_x' in train.columns:
            train['onpromotion'] = train['onpromotion_x'].fillna(0).astype(int)
            train.drop(columns=['onpromotion_x', 'onpromotion_'], errors='ignore', inplace=True)
        else:
            train['onpromotion'] = train['onpromotion'].fillna(0).astype(int)

        # Now encode 'family'
        train['family'] = train['family'].astype('category').cat.codes
    
        train['dayofweek'] = train['date'].dt.dayofweek
        train['month'] = train['date'].dt.month
        train['day'] = train['date'].dt.day
        train['week'] = train['date'].dt.isocalendar().week.astype(int)
        train['transactions'] = train['transactions'].fillna(0)

        # 4. Lag and Rolling Features
        train = train.sort_values(['store_nbr', 'family', 'date'])
        return train
    
    processed =preprocess(_df.copy(),_stores.copy(),_holidays.copy(),_transactions.copy())
    train = processed[processed['date'] < _separator]
    test = processed[processed['date'] >= _separator]
                      #test  =preprocess(_test.copy(),_stores.copy(),_holidays.copy(),_transactions.copy())
    
    return train,test

def preprocess_feature_dataset(dataset,train_func,train_name,mode):
   
    df=add_lag_rolling(dataset.copy(),train_func,train_name,mode=mode)
    for col in ['lag_21', 'lag_14', 'rolling_mean_21', 'rolling_std_21', 'rolling_mean_14', 'rolling_std_14']:
        df[col] = np.log1p(df[col])
    for col in ['trans_lag_21', 'trans_lag_14', 'trans_roll_mean_21', 'trans_roll_mean_14']:
        df[col] = np.log1p(df[col] + 1e-5)
    features = [
        'store_nbr', 'family', 'dayofweek', 'month', 'day', 'week', 'is_holiday',
        'transactions', 'lag_21', 'lag_14', 'trans_lag_21', 'trans_lag_14',
        'rolling_mean_21', 'rolling_std_21', 'rolling_mean_14', 'rolling_std_14',
        'trans_roll_mean_21', 'trans_roll_mean_14',

        #'store_dow', 
        #'family_month', 
        'onpromo_lag_21', 'onpromo_mean_14'
    ]
                             
    
    df = df[df[features].notna().all(axis=1)].copy()     
                             
 
    df[features].dropna(inplace=True)
                      
    X = df[features]
    y = np.log1p(df['sales'])
    if mode =="train":
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)
        return X_train,X_val,y_train,y_val
    else:
        return X,X,y,y
    
# START EXPERIENCE =====================================================
results = {}
preprocess_execution_tracker = ExecutionTimeTracker('preprocess_dataset')
train_df,test_df = preprocess_dataset()
preprocess_execution_tracker.done()                         
                             
for train_name, train_func in regimes.items():
    
    run_settings['regime'] = train_name
    if experiment_tracker.did_run(config,target_date,run_settings):
        continue
    else:
   
        print(train_name)
        # 1. Load Dataset

        print('Preprocessing features')
        preprocess_features_execution_tracker = ExecutionTimeTracker('preprocess_feature_dataset')  

        X_train,X_val,y_train,y_val= preprocess_feature_dataset(train_df.copy(),train_func,train_name,"train")

        _test_val_separator = int(len(X_val)/2)

        X_val_full = X_val.copy()
        y_val_full = y_val.copy()

        X_val = X_val_full.iloc[:_test_val_separator]
        X_test = X_val_full.iloc[_test_val_separator:]
    
        y_val = y_val_full.iloc[:_test_val_separator]
        y_test = y_val_full.iloc[_test_val_separator:]


        preprocess_features_execution_tracker.done()

        xgb_train_execution_tracker = ExecutionTimeTracker('xgb_train__1000')   

        model = xgb.XGBRegressor(
            n_estimators=run_settings["n_estimators"], 
            learning_rate=run_settings["learning_rate"], 
            max_depth=run_settings["max_depth"], 
            random_state=run_settings["random_state"])

        model.fit(X_train, y_train,
                  eval_set=[(X_val, y_val)],

                  verbose=25)

        xgb_train_execution_tracker.done()
        y_pred = np.expm1(model.predict(X_train))
        y_real = np.expm1(y_train)
        rmsle_train = np.sqrt(mean_squared_log_error(y_real, y_pred))

        # 6. Evaluation
        y_pred = np.expm1(model.predict(X_val))
        y_real = np.expm1(y_val)

        rmsle_val = np.sqrt(mean_squared_log_error(y_real, y_pred))

        y_pred = np.expm1(model.predict(X_test))
        y_real = np.expm1(y_test)

        rmsle_test = np.sqrt(mean_squared_log_error(y_real, y_pred))



        #results[train_name]=[rmsle]
        results[train_name]=[rmsle_train,rmsle_val,rmsle_test]

 
        experiment_tracker.save_run(
            config,
            {"train":rmsle_train,"val":rmsle_val,"test":rmsle_test},
            target_date,
            run_settings
        )
        print(f"RMSLE:{rmsle_train:.4f}, {rmsle_val:.4f},{rmsle_test:.4f}")


[ExecutionTimeTracker]::preprocess_dataset - Start.
[ExecutionTimeTracker]::preprocess_dataset - Complete. Took 4s to complete.

rolling_mean
Preprocessing features
[ExecutionTimeTracker]::preprocess_feature_dataset - Start.


C:\Users\Francisco Sá\AppData\Local\Temp\ipykernel_26160\2488502499.py:182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[features].dropna(inplace=True)


[ExecutionTimeTracker]::preprocess_feature_dataset - Complete. Took 497s to complete.

[ExecutionTimeTracker]::xgb_train__1000 - Start.
[0]	validation_0-rmse:2.94943
[25]	validation_0-rmse:0.99390
[50]	validation_0-rmse:0.59147
[75]	validation_0-rmse:0.52518
[100]	validation_0-rmse:0.50579
[125]	validation_0-rmse:0.49457
[150]	validation_0-rmse:0.48777
[175]	validation_0-rmse:0.47980
[200]	validation_0-rmse:0.47349
[225]	validation_0-rmse:0.46901
[250]	validation_0-rmse:0.46471
[275]	validation_0-rmse:0.46127
[300]	validation_0-rmse:0.45901
[325]	validation_0-rmse:0.45508
[350]	validation_0-rmse:0.45333
[375]	validation_0-rmse:0.45111
[400]	validation_0-rmse:0.44939
[425]	validation_0-rmse:0.44715
[450]	validation_0-rmse:0.44751
[475]	validation_0-rmse:0.44641
[500]	validation_0-rmse:0.44529
[525]	validation_0-rmse:0.44398
[550]	validation_0-rmse:0.44388
[575]	validation_0-rmse:0.44309
[600]	validation_0-rmse:0.44247
[625]	validation_0-rmse:0.44184
[650]	validation_0-rmse:0.44101
[675]

C:\Users\Francisco Sá\AppData\Local\Temp\ipykernel_26160\2488502499.py:182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[features].dropna(inplace=True)


[ExecutionTimeTracker]::preprocess_feature_dataset - Complete. Took 668s to complete.

[ExecutionTimeTracker]::xgb_train__1000 - Start.
[0]	validation_0-rmse:2.94932
[25]	validation_0-rmse:0.98895
[50]	validation_0-rmse:0.57529
[75]	validation_0-rmse:0.51446
[100]	validation_0-rmse:0.49689
[125]	validation_0-rmse:0.48738
[150]	validation_0-rmse:0.48059
[175]	validation_0-rmse:0.47263
[200]	validation_0-rmse:0.46674
[225]	validation_0-rmse:0.46388
[250]	validation_0-rmse:0.46181
[275]	validation_0-rmse:0.45928
[300]	validation_0-rmse:0.45912
[325]	validation_0-rmse:0.45861
[350]	validation_0-rmse:0.46001
[375]	validation_0-rmse:0.46025
[400]	validation_0-rmse:0.46079
[425]	validation_0-rmse:0.46219
[450]	validation_0-rmse:0.46172
[475]	validation_0-rmse:0.46082
[500]	validation_0-rmse:0.46125
[525]	validation_0-rmse:0.46131
[550]	validation_0-rmse:0.46011
[575]	validation_0-rmse:0.45984
[600]	validation_0-rmse:0.45851
[625]	validation_0-rmse:0.45734
[650]	validation_0-rmse:0.45800
[675]

C:\Users\Francisco Sá\AppData\Local\Temp\ipykernel_26160\2488502499.py:182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[features].dropna(inplace=True)


[ExecutionTimeTracker]::preprocess_feature_dataset - Complete. Took 577s to complete.

[ExecutionTimeTracker]::xgb_train__1000 - Start.
[0]	validation_0-rmse:2.94789
[25]	validation_0-rmse:0.97262
[50]	validation_0-rmse:0.55940
[75]	validation_0-rmse:0.49652
[100]	validation_0-rmse:0.47886
[125]	validation_0-rmse:0.47072
[150]	validation_0-rmse:0.46422
[175]	validation_0-rmse:0.45856
[200]	validation_0-rmse:0.45385
[225]	validation_0-rmse:0.44905
[250]	validation_0-rmse:0.44713
[275]	validation_0-rmse:0.44399
[300]	validation_0-rmse:0.44044
[325]	validation_0-rmse:0.43894
[350]	validation_0-rmse:0.43589
[375]	validation_0-rmse:0.43358
[400]	validation_0-rmse:0.43175
[425]	validation_0-rmse:0.42901
[450]	validation_0-rmse:0.42701
[475]	validation_0-rmse:0.42513
[500]	validation_0-rmse:0.42296
[525]	validation_0-rmse:0.42194
[550]	validation_0-rmse:0.42050
[575]	validation_0-rmse:0.41958
[600]	validation_0-rmse:0.41878
[625]	validation_0-rmse:0.41770
[650]	validation_0-rmse:0.41635
[675]

C:\Users\Francisco Sá\AppData\Local\Temp\ipykernel_26160\2488502499.py:182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[features].dropna(inplace=True)


[ExecutionTimeTracker]::preprocess_feature_dataset - Complete. Took 2884s to complete.

[ExecutionTimeTracker]::xgb_train__1000 - Start.
[0]	validation_0-rmse:2.94917
[25]	validation_0-rmse:0.99416
[50]	validation_0-rmse:0.59289
[75]	validation_0-rmse:0.52797
[100]	validation_0-rmse:0.50964
[125]	validation_0-rmse:0.49935
[150]	validation_0-rmse:0.49065
[175]	validation_0-rmse:0.48325
[200]	validation_0-rmse:0.47600
[225]	validation_0-rmse:0.46826
[250]	validation_0-rmse:0.46503
[275]	validation_0-rmse:0.46197
[300]	validation_0-rmse:0.45996
[325]	validation_0-rmse:0.45695
[350]	validation_0-rmse:0.45441
[375]	validation_0-rmse:0.45319
[400]	validation_0-rmse:0.45149
[425]	validation_0-rmse:0.45026
[450]	validation_0-rmse:0.44812
[475]	validation_0-rmse:0.44543
[500]	validation_0-rmse:0.44377
[525]	validation_0-rmse:0.44163
[550]	validation_0-rmse:0.43944
[575]	validation_0-rmse:0.43724
[600]	validation_0-rmse:0.43601
[625]	validation_0-rmse:0.43470
[650]	validation_0-rmse:0.43505
[675

### What’s a Good Score?
For Store Sales (Kaggle):

🥉 RMSLE > 0.60 → Baseline or simple model

🥈 RMSLE ≈ 0.45 → Reasonable with lags + rolling + calendar features

🥇 RMSLE < 0.40 → Competitive (you’re doing very well)

🏆 RMSLE < 0.38 → Likely leaderboard top 10%



In [ ]:
results

In [ ]:
#[7,14]
prev_results = {'rolling_mean': [0.39173891701038477,
  0.39667409264677866,
  0.38245842805848546],
 'rolling_median': [0.3974037983694567,
  0.40459857824155593,
  0.3850072307441535],
 'rolling_weighted_mean': [0.38756927881069714,
  0.38920871379602145,
  0.38959106223497975],
 'rolling_winsorized_mean': [0.39197364429035875,
  0.3960417851284805,
  0.3797409070979486]}

def print_diffs(metric):
    print(metric)
    diff = abs(prev_results[metric][0]-prev_results[metric][1])+abs((prev_results[metric][0]-prev_results[metric][2]))#+abs((prev_results[metric][1]-prev_results[metric][2]))
    print(diff/2,abs((prev_results[metric][0]-prev_results[metric][2])))
    
print_diffs('rolling_mean')
print_diffs('rolling_median')
print_diffs('rolling_weighted_mean')
print_diffs('rolling_winsorized_mean')

In [ ]:
"""
{'rolling_mean': [0.39173891701038477,
  0.39667409264677866,
  0.38245842805848546],
 'rolling_median': [0.3974037983694567,
  0.40459857824155593,
  0.3850072307441535],
 'rolling_weighted_mean': [0.38756927881069714,
  0.38920871379602145,
  0.38959106223497975],
 'rolling_winsorized_mean': [0.39197364429035875,
  0.3960417851284805,
  0.3797409070979486]}

"""

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(8,6))

sns.heatmap(
    pd.DataFrame(prev_results),
    #er=0,
    
    annot=True, fmt=".1f", cmap="RdYlGn", 
    linewidths=0.5, linecolor='black'
           )
#plt.title("Train/Test Regime Mean Total Reward\n(rows=train, cols=test)")
plt.xlabel("Test Regime")
plt.ylabel("Train Regime")
plt.show()


In [ ]:
pd.DataFrame(results)